In [10]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

MODEL_PATH = 'efficientdet_lite2.tflite'
VIDEO_PATH = 'Snooker2.mp4'
MIN_SCORE = 0.009

def main():
    # 1. Konfiguracja MediaPipe
    base_options = python.BaseOptions(model_asset_path=MODEL_PATH)
    options = vision.ObjectDetectorOptions(
        base_options=base_options,
        score_threshold=MIN_SCORE,
        # Zostawiamy sports ball, ale to nadal loteria
        category_allowlist=['sports ball']
    )
    detector = vision.ObjectDetector.create_from_options(options)

    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print(f"Błąd: Nie można otworzyć wideo: {VIDEO_PATH}")
        return

    print("Naciśnij 'q', aby zakończyć.")

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        h, w, c = frame.shape

        margin_y = int(h * 0.15) # Obcinamy 15% z góry i dołu
        margin_x = int(w * 0.05) # Obcinamy 5% z boków

        roi_y_start = margin_y
        roi_y_end = h - margin_y
        roi_x_start = margin_x
        roi_x_end = w - margin_x

        # Tworzymy wycinek (sam stół)
        cropped_frame = frame[roi_y_start:roi_y_end, roi_x_start:roi_x_end]

        # Zabezpieczenie, gdyby wycinek był pusty
        if cropped_frame.size == 0:
            continue

        image_rgb_cropped = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB)
        mp_image_cropped = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb_cropped)

        # 3. Detekcja na wycinku
        detection_result = detector.detect(mp_image_cropped)

        annotated_frame = frame.copy()

        cv2.rectangle(annotated_frame, (roi_x_start, roi_y_start), (roi_x_end, roi_y_end), (255, 0, 0), 2)

        for detection in detection_result.detections:
            bbox = detection.bounding_box

            # Przeliczanie współrzędnych (tak jak wcześniej)
            start_x = bbox.origin_x + margin_x
            start_y = bbox.origin_y + margin_y
            end_x = start_x + bbox.width
            end_y = start_y + bbox.height


            box_width = bbox.width
            box_height = bbox.height


            if box_width > 50 or box_height > 50:
                continue

            if box_width < 5 or box_height < 5:
                continue


            ratio = box_width / box_height
            if ratio < 0.5 or ratio > 2.0:
                continue
            start_point = (start_x, start_y)
            end_point = (end_x, end_y)

            category = detection.categories[0]
            score = category.score
            category_name = category.category_name

            if category_name == 'sports ball':
                # Rysujemy zieloną ramkę
                cv2.rectangle(annotated_frame, start_point, end_point, (0, 255, 0), 2)
                # Podpis (skrócony, żeby nie zasłaniał)
                cv2.putText(annotated_frame, f"{score:.2f}", (start_x, start_y - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

        # 5. Wyświetlanie
        cv2.imshow('MediaPipe - Billiard Balls (ROI Fix)', annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

Naciśnij 'q', aby zakończyć.
